## 日志总结

提供日志总结功能，总结程序员最近在干什么事，干的内容是什么，具体包括：

1. **工作模式**：调用模式识别模块
2. **聚焦位置**：程序员当前关注的操作件或命令，来源于artifact_history和cmd_history

| 方案                                    | 输入                   | 输出          |
| --------------------------------------- | ---------------------- | ------------- |
| 聚焦位置直接选用频度最高的artifact和cmd | 预处理过的动作数据序列 | 模式+聚焦位置 |

存在的问题：

1. 移动文件功能无效
2. 创建文件夹会被记录为文件创建
3. 为什么有 SelectText 没有记录 hierarchy？（牢鼠-写python-0.2.2 id194）

In [200]:
import json
import enum
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from urllib.parse import unquote

import openai
import ollama

In [201]:
class EventTypeFileLevel(enum.Enum):
    """
    文件级事件，对文件进行操作的事件
    """
    OpenTextDocument = "OpenTextDocument"
    CloseTextDocument = "CloseTextDocument"
    ChangeTextDocument = "ChangeTextDocument"
    CreateFile = "CreateFile"
    DeleteFile = "DeleteFile"
    SaveFile = "SaveFile"
    RenameFile = "RenameFile"
    MoveFile = "MoveFile"

class EventTypeTextLevel(enum.Enum):
    """
    文本级事件，对文本进行操作的事件
    """
    AddTextDocument = "AddTextDocument"
    DeleteTextDocument = "DeleteTextDocument"
    EditTextDocument = "EditTextDocument"
    RedoTextDocument = "RedoTextDocument"
    UndoTextDocument = "UndoTextDocument"
    SelectText = "SelectText"
    MouseHover = "MouseHover"

class EventTypeTerminalLevel(enum.Enum):
    """
    终端级事件，在终端进行的事件
    """
    OpenTerminal = "OpenTerminal"
    CloseTerminal = "CloseTerminal"
    ChangeActiveTerminal = "ChangeActiveTerminal"
    ExecuteTerminalCommand = "ExecuteTerminalCommand"

In [202]:
def getTopLevelEventType(event_type: str) -> int:
    """
    根据事件类型获取顶层事件类型，包括：文件级、文本级、终端级和其他事件类型，
    分别赋予从 0 到 3 的顺序编号
    """
    for event_type_f in EventTypeFileLevel:
        if event_type_f.value == event_type:
            return 0
    for event_type_t in EventTypeTextLevel:
        if event_type_t.value == event_type:
            return 1
    for event_type_te in EventTypeTerminalLevel:
        if event_type_te.value == event_type:
            return 2
    return 3

In [203]:
path = '../dataset_raw/牢鼠-写python-0.2.2/2025-01-23 04.58.17.json'
# path = '../dataset_raw/于小丘-创建SQLModel数据库模型1-0.2.2/2025-01-20 22.23.14.json'
with open(path, 'r') as f:
    log_data = json.load(f)
print(len(log_data))

850


In [204]:
# 用于记录事件类型数量
dict_event_type = {}

"""
用于构建文件变化树（包含新建和删除的文件）
重命名的文件目前插件没有记录更改后的名称
移动的文件目前插件检测不到
"""
file_change_tree = {}

# 用于记录工件结构树
artifact_tree = {}

# 精简后的 log 记录
refined_log = []

In [205]:
def process_change_file(file_path, info):
    global file_change_tree
    file_path_list = unquote(file_path).split('/')
    while '' in file_path_list:
        file_path_list.remove('')
    cur_dict = file_change_tree
    for path in file_path_list[:-1]:
        if path not in cur_dict or not isinstance(cur_dict[path], dict):
            cur_dict[path] = {}
        cur_dict = cur_dict[path]
    cur_dict[file_path_list[-1]] = info


def build_artifact_tree(hierarchy):
    global artifact_tree
    # 结构第一层必须是文件路径
    assert hierarchy[0]['type'] == 'File', f"{hierarchy[0]['type']} is not File"
    
    file_path_list = unquote(hierarchy[0]['name']).split('/')
    while '' in file_path_list:
        file_path_list.remove('')
    
    cur_dict = artifact_tree
    for path in file_path_list: # 构建文件路径
        if path == file_path_list[-1]:
            path = f'{path}: File'
        elif path[-1] != ':':
            path = f'{path}: Folder'
        if path not in cur_dict:
            cur_dict[path] = {}
        cur_dict = cur_dict[path]

    for artifact in hierarchy[1:]: # 构建文件内结构
        path = f"{artifact['name']}: {artifact['type']}"
        if path not in cur_dict:
            cur_dict[path] = {}
        cur_dict = cur_dict[path]
        if '*visit' in cur_dict:
            cur_dict['*visit'] += 1
        else:
            cur_dict['*visit'] = 1

In [206]:
top_level_cnt = [0,0,0,0]

for log in log_data:
    log_type = log['eventType']

    if log_type in dict_event_type:
        dict_event_type[log_type] += 1
    else:
        dict_event_type[log_type] = 1
    
    top_level_type = getTopLevelEventType(log_type)
    top_level_cnt[top_level_type] += 1
    
    if top_level_type == 0:
        if log_type == 'CreateFile':
            process_change_file(log['artifact']['name'], 'create')
        elif log_type == 'DeleteFile':
            process_change_file(log['artifact']['name'], 'delete')
    elif top_level_type == 1:
        if 'hierarchy' in log['artifact']:
            build_artifact_tree(log['artifact']['hierarchy'])



In [207]:
list_event_type = list(dict_event_type.items())
list_event_type.sort(key=lambda x:x[1],reverse=True)
for event in list_event_type:
    print(event)

('EditTextDocument', 290)
('SelectText', 165)
('AddTextDocument', 165)
('SaveFile', 92)
('DeleteTextDocument', 41)
('CreateFile', 37)
('ChangeTextDocument', 19)
('UndoTextDocument', 16)
('MouseHover', 7)
('ChangeActiveTerminal', 6)
('DeleteFile', 6)
('OpenTerminal', 3)
('CloseTerminal', 3)


In [208]:
print(json.dumps(file_change_tree,indent=4))

{
    "file:": {
        "d:": {
            "Code": {
                "Python": {
                    "KON": {
                        "main.py": "create",
                        "Vue": {
                            "README.md": "create",
                            "env.d.ts": "create",
                            "index.html": "create",
                            "package.json": "create",
                            "public": {
                                "favicon.ico": "create"
                            },
                            "src": {
                                "App.vue": "create",
                                "assets": {
                                    "base.css": "delete",
                                    "logo.svg": "delete",
                                    "main.css": "create"
                                },
                                "components": {
                                    "HelloWorld.vue": "delete",
                      

In [209]:
print(json.dumps(artifact_tree,indent=4))

{
    "file:": {
        "d:": {
            "Code: Folder": {
                "Python: Folder": {
                    "KON: Folder": {
                        "Vue: Folder": {
                            "tsconfig.app.json: File": {
                                "extends: String": {
                                    "*visit": 1
                                }
                            },
                            "vite.config.ts: File": {},
                            "index.html: File": {
                                "html: Field": {
                                    "*visit": 60,
                                    "head: Field": {
                                        "*visit": 59,
                                        "title: Field": {
                                            "*visit": 58
                                        }
                                    }
                                }
                            },
                            

In [210]:
def getFileChangeTreeWithLabel(dict_tree):
    if not dict_tree: return [], [], []
    parents = []
    nodes = []
    info = []
    label_index = 0
    indices = []
    # 使用 BFS 实现
    cur_dict_tree = dict_tree
    while True:
        key = list(cur_dict_tree.keys())[0]
        if isinstance(cur_dict_tree[key], dict) and len(cur_dict_tree[key].keys()) == 1:
            cur_dict_tree = cur_dict_tree[key]
        else: break
    node_queue = list(cur_dict_tree.keys())
    dict_queue = [cur_dict_tree[key] for key in cur_dict_tree.keys()]
    indices = list(range(len(cur_dict_tree.keys())))
    label_index += len(cur_dict_tree.keys())
    while node_queue:
        parent_node = node_queue.pop(0)
        parent_dict = dict_queue.pop(0)
        index = indices.pop(0)
        # print(index,parent_node,parent_dict)
        if not isinstance(parent_dict, dict): continue
        node_queue.extend(parent_dict.keys())
        dict_queue.extend([parent_dict[key] for key in parent_dict.keys()])
        for key in parent_dict.keys():
            nodes.append(f'{key} - {label_index}')
            parents.append(f'{parent_node} - {index}')
            indices.append(label_index)
            label_index += 1
            if isinstance(parent_dict[key], dict):
                info.append('')
            else:
                info.append(parent_dict[key]) 
    return nodes, parents, info


def getArtifactTreeWithLabel(dict_tree):
    parents = []
    nodes = []
    label_index = 0
    indices = []
    info = []
    # 使用 BFS 实现
    cur_dict_tree = dict_tree
    while True:
        key = list(cur_dict_tree.keys())[0]
        if isinstance(cur_dict_tree[key], dict) and len(cur_dict_tree[key].keys()) == 1:
            cur_dict_tree = cur_dict_tree[key]
        else: break
    node_queue = list(cur_dict_tree.keys())
    dict_queue = [cur_dict_tree[key] for key in cur_dict_tree.keys()]
    indices = list(range(len(cur_dict_tree.keys())))
    label_index += len(cur_dict_tree.keys())
    while node_queue:
        parent_node = node_queue.pop(0)
        parent_dict = dict_queue.pop(0)
        index = indices.pop(0)
        # print(index,parent_node,parent_dict)
        if not isinstance(parent_dict, dict): continue
        node_queue.extend(parent_dict.keys())
        dict_queue.extend([parent_dict[key] for key in parent_dict.keys()])
        for key in parent_dict.keys():
            if key == '*visit':
                indices.append('')
                continue
            nodes.append(f'{key} - {label_index}')
            parents.append(f'{parent_node} - {index}')
            indices.append(label_index)
            label_index += 1
            info.append(parent_dict[key].get('*visit', 0))
    return nodes, parents, info

In [211]:
nodes, parents, info = getFileChangeTreeWithLabel(file_change_tree)
fig = go.Figure(go.Treemap(
    labels = nodes,
    parents = parents,
    hovertemplate="""<b>%{label}</b><br>
    Parent: %{parent}<br>
    Change: %{customdata}
    <extra></extra>""",
    customdata=info,
    root_color="lightgrey"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

In [212]:
nodes, parents, info = getArtifactTreeWithLabel(artifact_tree)
vals = info
for i in range(len(vals)):
    vals[i] += 20
fig = go.Figure(go.Treemap(
    labels = nodes,
    parents = parents,
    hovertemplate="""<b>%{label}</b><br>
    Parent: %{parent}<br>
    Visit: %{customdata}
    <extra></extra>""",
    customdata=info,
    # values=vals,
    root_color="lightgrey"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.update_traces(marker=dict(cornerradius=5))
fig.show()
fig.write_html("artifact_tree.html")

In [213]:
def getLimitedContent(content: str, max_length: int = 64):
    if len(content) > max_length:
        return content[:max_length] + "..."
    else:
        return content

def getRefinedLogItem(log_item: dict) -> dict:
    refined_log_item = {}
    top_level_type = getTopLevelEventType(log_item["eventType"])
    refined_log_item["event"] = log_item["eventType"]
    if top_level_type == 0:
        refined_log_item["path"] = unquote(log_item["artifact"]["name"])
    elif top_level_type == 1:
        if "hierarchy" in log_item["artifact"]:
            refined_log_item["path"] = unquote(log_item["artifact"]["hierarchy"][0]["name"])
            refined_log_item["action"] = log_item["context"]["type"]
            refined_log_item["contentBefore"] = getLimitedContent(log_item["context"]["content"]["before"])
            refined_log_item["contentAfter"] = getLimitedContent(log_item["context"]["content"]["after"])
    # elif top_level_type == 2:
    #     refined_log_item["path"] = "termial:" + log_item["artifact"]["name"]
    return refined_log_item

for log in log_data:
    refined_log.append(getRefinedLogItem(log))

latest_200_log = json.dumps(refined_log[-200:], indent=4)
print(latest_200_log)

[
    {
        "event": "SaveFile",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/Header.vue"
    },
    {
        "event": "CreateFile",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue"
    },
    {
        "event": "AddTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Add",
        "contentBefore": "",
        "contentAfter": "<"
    },
    {
        "event": "AddTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Add",
        "contentBefore": "",
        "contentAfter": "te"
    },
    {
        "event": "EditTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Edit",
        "contentBefore": "te",
        "contentAfter": "template"
    },
    {
        "event": "AddTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/view

In [214]:
full_prompt = f"""
以下是收集的开发者在 IDE 内的动作数据日志，动作包括：
1. 对文件的操作（打开、关闭、新建、删除等）
2. 对具体文本内容的操作（新增、删除、选择、鼠标悬停等）

对日志进行总结，无需输出代码。
总结：开发者最近对项目进行了哪些修改，主要集中在哪些文件，修改的内容是什么。

```json
{latest_200_log}
```
"""

print(full_prompt)


以下是收集的开发者在 IDE 内的动作数据日志，动作包括：
1. 对文件的操作（打开、关闭、新建、删除等）
2. 对具体文本内容的操作（新增、删除、选择、鼠标悬停等）

对日志进行总结，无需输出代码。
总结：开发者最近对项目进行了哪些修改，主要集中在哪些文件，修改的内容是什么。

```json
[
    {
        "event": "SaveFile",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/Header.vue"
    },
    {
        "event": "CreateFile",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue"
    },
    {
        "event": "AddTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Add",
        "contentBefore": "",
        "contentAfter": "<"
    },
    {
        "event": "AddTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Add",
        "contentBefore": "",
        "contentAfter": "te"
    },
    {
        "event": "EditTextDocument",
        "path": "file:///d:/Code/Python/KON/Vue/src/components/views/About.vue",
        "action": "Edit",
        "contentBefore": "te",
 

In [215]:
stream = ollama.chat(
  model='qwen2.5',
  messages=[{'role': 'user', 'content': full_prompt}],
  stream=True,
)
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

从日志记录来看，你在修改Vue项目的路由配置文件 `router.ts`。具体地，你添加了一个名为 `'About'` 的路由，并且这个路由指向了 `components/views/About.vue` 组件。

下面是你的操作步骤总结：

1. 打开或编辑 `src/router.ts` 文件。
2. 添加一个新的路由条目：
   ```typescript
   {
       path: '/about',
       name: 'About',
       component: () => import('../components/views/About.vue')
   }
   ```

这将添加一个路径为 `/about` 的路由，其组件是 `components/views/About.vue`。

3. 保存并关闭文件。

具体操作步骤如下：
1. 打开 `src/router.ts` 文件。
2. 在适当的位置（通常在所有其他路由条目之后）插入以下代码：
   ```typescript
   {
       path: '/about',
       name: 'About',
       component: () => import('../components/views/About.vue')
   }
   ```
3. 保存文件。

现在，当你访问 `/about` 路径时，将会渲染 `components/views/About.vue` 组件。这个操作确保了你的路由配置是完整的，并且可以支持新的页面或功能。

如果你需要进一步的帮助，请告知具体需求。

In [216]:
stream = ollama.chat(
  model='deepseek-r1',
  messages=[{'role': 'user', 'content': full_prompt}],
  stream=True,
)
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

<think>
嗯，用户给了一个JSON数组，里面有很多事件记录。看起来这些事件是关于某个应用程序的活动日志。我需要先理解这个数据结构，看看每个对象包含哪些字段。

首先，每个对象都有`event`、`path`和`action`等字段。比如第一个事件是`{"event": "AddTextDocument", "path": "...", "action": "Add"}`，这说明发生了添加文本文档的操作，并且路径指向了`src/router.ts`。

接下来，我注意到有很多类似的操作，比如反复的编辑文本、保存文件等等。这些可能是在测试一个功能或者自动化脚本运行时记录的事件。用户可能希望从这些日志中提取有用的信息，或者分析应用程序的行为模式。

用户的请求有点模糊，只提供了这段数据和一些字段名。可能需要更多的上下文来确定具体的需求是什么。比如是否需要统计某种事件的数量、找出特定操作的时间点，还是生成某个报告？

另外，我看到路径里包含了`src/`, `components/views/`, 这表明可能涉及到组件的加载或修改。这可能与 Vue.js 的动态路由有关。

总结一下，用户的问题涉及分析一个包含应用程序事件的日志数据，并且需要明确他们的具体需求是什么。可能需要进一步询问来确定是否需要特定类型的分析或者处理。
</think>

看起来这是某段代码或日志记录，但文件名和内容不完整，无法直接识别具体用途或功能。如果您能提供更多的背景信息或完整的文件内容，我可以帮助您分析或解释这段代码的功能。

以下是针对部分数据的初步观察：

1. **事件类型**：
   - 包括`AddTextDocument`、`ChangeTextDocument`、`EditTextDocument`等操作。
   - 可能涉及到文本编辑、文档创建、加载组件等操作。

2. **路径信息**：
   - `path`字段指向`src/router.ts`和`components/views/About.vue`，可能与项目结构相关。
   - 代码中包含`src/`, `components/views/`等子目录，可能涉及动态路由或组件加载。

3. **事件操作**：
   - 添加文本内容、保存文件、编辑文本内容等。
   - 可能涉及到单元测试或自动化脚本运行时的事件捕